# Cryptographic Utilities

## Exercise 1: MD5, SHA1, SHA256, SHA512

Generate the previsously mentioned hash values of the string `Hello World!`. Do You get the same result as Your neighbour? Check the size of each hash value! What are the corresponding block sizes?
Make a table! How many hash values would You have to generate in order to find a collision with a probability of at least 50% (90%)?

## Exercise 2: True random number generator

Check if Your computer can generate true random numbers, possibly through some special device? Find out the underlying physical principle (thermal noise, etc.). How can You use the random number generator in the power shell (Windows) or some shell (bash, sh, etc.) on Linux or MAC OS X?
How are true random numbers generated on smart cards? Do some research on the internet!

#### True random numbers:
- The /dev/random device on Linux: Generates random numbers, “blocks” and doesn’t return a result until it gathers enough entropy to return a truly random number.
- Intel chips include a hardware-based random number generator known as RdRand. This chip uses an entropy source on the processor and provides random numbers to software when the software requests them. The problem here is that the random number generator is essentially a black box and we don’t know what’s going on inside it.
- Sources of randomness from the environment include 
 - inter-keyboard timings, 
 - inter-interrupt timings from some interrupts, 
 - and other events which are both 
   - (a) non-deterministic and 
   - (b) hard for an outside observer to measure.  
- Randomness from these sources are added to an "entropy pool", which is mixed using a CRC-like function.

#### PowerShell
- Get-Random sets a default seed for each session based on the system time clock when the session starts.

#### Smart Cards
Current random numbergenerators in the smart card micro-controllers are generallybased on Linear Feedback Shift Registers (LFSR) driven byvoltage-controlled oscillators.

Source: http://www.academia.edu/2519923/Pseudorandom_Number_Generation_in_Smart_Cards_An_Implementation_Performance_and_Randomness_Analysis

## Exercise 3: Linear congruential random number generator

Compute the period of a **linear congruential number** generator if $m = 11$, $b = 5$, $a = 3$ and $x_0 = 7$ is used. Write down the sequence of random number! What is the maximal size of the period? Adapt the parameters, such that the period is maximal!

In [86]:
def lcp(m, a, b, x):
    periodsize = 0
    numbers = []
    def lcp_inner(m, a, b, x):
        nonlocal periodsize
        nonlocal numbers
        random = (a*x+b)%m
        if random not in numbers:
            numbers.append(random)
            periodsize += 1
            lcp_inner(m, a, b, random)
        else:
            print("size of period: " + str(periodsize))
            print("randomnumbers: " + str(numbers))
    lcp_inner(m, a, b, x)

In [91]:

## with parameters
lcp(11,3,5,7)


print("\nwith maximal size of period")
## a must be 1 and b between 2 and 10
lcp(11,1,5,7)

size of period: 5
randomnumbers: [4, 6, 1, 8, 7]

with maximal size of period
size of period: 11
randomnumbers: [1, 6, 0, 5, 10, 4, 9, 3, 8, 2, 7]


## Exercise 4a: LFSR

Construct an LFSR of length $5$ which produces the sequence of bits $(1, 0, 1, 1, 0, 0, 1, 0, 1, 0,...)$. What is the period of this LFSR? What is the maximal period? Try to improve this LFSR such that it has maximum period!

**Solution**: Connection (or feedback) polynomial $C(D) = 1 + D + D^3 + D^5$; recursive formula
$$ s_j =
\sum_{k=1}^5{
c_k s_{j−k}} \text{mod}\ 2= s_{j−1}+s_{j−3}+s_{j−5}\text{mod}\ 2, j ≥ 5.$$ 
Period is $15$ which can be improved to $31$ by adding $c_2 = 1$ as an example! To make computations easy, use the following python code
```python
def lfsr5(seed, taps):
	sr, xor = seed, 0
	while 1:
		for t in taps:
			xor += int(sr[t − 1])
		xor %= 2
		sr, xor = str(xor) + sr[: − 1], 0
		yield sr
		if sr == seed:
			break

a = lfsr5(’10110’, (5,3,2,1))
[next(a) for j in range(31)]
```

## Exercise 4b: Is random really random (optional)
Read section 5.4.4 in the _Handbook of Applied Cryptography_ by A. Menezes, P. van Oorschot and S. Vanstone (p 181, 182, 183). Here we will only look at the monobit and the two-bit test of example 5.31 in the same book.
Consider the (probably non-random) sequence $s$ of length $n = 160$ obtained by replicating the following sequence four times

$$11100\ 01100\ 01000\ 10100\ 11101\ 11100\ 10010\ 01001$$

Apply the monobit and two-bit test. Based on these two tests, would you say, the sequence random?


**Solution**: monobit test passed because: $X_1 = 0.4 < 3.8415$; two-bit test passed because: $X_2 = 0.6252 <3.8415$; sequence would also pass poker test, but not runs and autocorrelation test.

## Exercise 5: Probability for a collision
There are 40 people in a room. You bet, that there are at least 2 people with the same birthday. What is the probability, that You win? Use the exact formula as well as the approximation!

**Solution**: Probility of having two people with the same birthday is $0.882$ or $88.2\%$. Use symbolic python for the exact answer:
```python
from sympy import *
i = symbols(’i’,integer=True)
p = 1 − product((365 − i)/365,(i,1,39))
p.evalf()
```

Probability of at least 2 people having the same birthday = 1 - probability of all people having different birthdays

probability of all people having different birthdays = $\frac{\prod_{n=0}^{39}(365-n)}{365^{40}} = 0.1088$

Probability of at least 2 people having the same birthday = $1 - 0.1088 = 0.8912$

## Exercise 6: Probability of a collision

Suppose You use a hash function of length $128$ bits. How many hash values would you have to compute in order to find a collision with probability at least 90%? Use the approximate forumula from Slide 14/58:

$$n \approx 2^{(m+1)/2}\sqrt{\ln\left(\frac{1}{1−p}\right)}$$

where $m = 128$ and $p = 0.9$. How much storage would you need, if you need $16$ bytes for each hash?

**Solution**: $n \approx 3.95861\cdot10^{19}$ ; $633'377'600$ terabytes storage.